# Navigation Recognition

The idea is on given element from the DOM tree of a webpage to determine if this particular element is the main navigation of the page. By main navigation we define the only one navigation with the main section or category links. Every page has a main navigation and can contains multiple menus, context navigations and so on. We are trying to determine onlt the main navigation.

## Linear regression statistic model

Using weights on element DOM information we calculate probability. After that we build a linear model from the element based on accessibility roles, HTML tag, class names, ids and other components that we will cover later.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
from IPython.display import display

%matplotlib notebook

import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")
warnings.filterwarnings(action="ignore", module="sklearn", message="^Objective did not")

In [2]:
df = pd.read_csv("linear-regression-input.csv", header=0)
df = df.drop_duplicates(subset=['url', 'tag', 'class', 'id', 'role', 'home', 'depth', 'links', 'links_by_depth'], keep='first')

Y = df['probability']
X = df[['tag', 'class', 'id', 'role', 'home', 'depth', 'links', 'links_by_depth']]

## Understand the model

The linear model of every element is based on several criteria

### HTML tag

| tag    	| weight    | comment 	|
| --------	| --------	| ---------	|
| `nav`   	| **3**    	| we are sure that this element holds some navigation 	|
| `div`    	| **2**    	| they are many containers holding the navigation itself 	|
| `header` 	| **1**    	| it is common to have navigation in the header 	|
| other  	| **0**    	| we are neutral here, cannot tell 	|
| `footer` 	| **-1**   	| if it is footer, we really don't care of it 	|

### Element classes and ids

| weight 	| keywords            	| comment                                                                               	|
|--------	|---------------------	|---------------------------------------------------------------------------------------	|
| **2**  	| `navigation`, `nav` 	| we assume that this can be navigation or container for sure if these keywords appears 	|
| **1**  	| `menu`, `main`      	| we are not sure if this can be main navigation but could be in some corner cases      	|
| **-1** 	| `footer`            	| keywords indicating that we are not interested in this element for sure               	|

### Accessibility roles

| role                         	| priority 	| comment                                          	|
|------------------------------	|----------	|--------------------------------------------------	|
| `navigation`                 	| **high** 	| for sure this is the main navigation on the page 	|
| `menu`, `menubar`, `toolbar` 	| medium   	| menus, context covigations and etc in most cases  |
| `contentinfo`                	| low      	| common used for the footer                       	|

### Other
- `depth` - Depth of the element in the DOM tree
- `home` - Whatever if element has links and some of them points to the home ('/')
- `links` - if the element contains links
- `links_by_depth` - if more than half of the links are at same depth on the DOM tree

## Probability
We calculate some probability using weights on the same criteria for every element. Using this probability we try to find a successful rate of the linear regression model.

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)  

model = LinearRegression()

model.fit(X_train, y_train) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

## Coefficients

Using our data set lets examine that are the coefficient that every criteria participates. For example you will se that `depth` coefficient is negative, but if we train the model without it the score drop with 0.01% This leads me to the conclustion that element deeper in the DOM tree are less possible to be navigation.

In [10]:
coeff_df = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])  
coeff_df

,Coefficient
tag,0.056127
class,0.189682
id,0.071158
role,0.267551
home,0.357157
depth,-0.003405
links,0.047381
links_by_depth,0.221812


## Score of the current model

In [11]:
model.score(X, Y)

0.9258316624431402

In [12]:
y_pred = model.predict(X_test) 
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})  
df

,Actual,Predicted
1675,0.40,0.351993
314,0.40,0.453794
86,0.85,0.819538
128,0.15,0.153959
2494,0.15,0.157364
303,0.15,0.177792
1435,0.15,0.160768
21,0.15,0.164173
1501,0.25,-0.060105
1427,0.40,0.440175


In [13]:
from sklearn import metrics  
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

Mean Absolute Error: 0.05731773645475403
Mean Squared Error: 0.00785394093604449
Root Mean Squared Error: 0.08862246293149662


## Testing

Using https://www.rheinische-anzeigenblaetter.de/ our library recognized successfully the page header, footer and navigation with the following internal probability recognized as the main navigation:

- `navigation` (1.4) with model `"3,2,0,0,1,4,1,1"`
- `header` (0.6) with model `"1,0,0,0,1,3,1,1"`
- `footer` (0.25) with model `"-1,-1,0,0,0,3,1,1"`

Now lets check how our linear regression will score with these models

In [14]:
array = np.array([[3,2,0,0,1,4,1,1], [1,0,0,0,1,3,1,1], [-1,-1,0,0,0,3,1,1]])
model.predict(array)

array([1.24644046, 0.75822812, 0.09913631])

## Results

As we see the linear regression model predicts on random data pretty nice. We can assume from the results that the navigation is inside the header. Our library also confirm this.